# **Tipos de Saída**

### **Argumentos e Descrição**

| **Argumento** | **Descrição** |
|--------------|--------------|
| `n_var` | Valor inteiro representando o número de variáveis de decisão. |
| `n_obj` | Valor inteiro representando o número de objetivos. |
| `n_constr` | Valor inteiro representando o número de restrições. |
| `xl` | Float ou `np.ndarray` de tamanho `n_var`, representando os limites inferiores das variáveis de decisão. |
| `xu` | Float ou `np.ndarray` de tamanho `n_var`, representando os limites superiores das variáveis de decisão. |
| `vtype` | (Opcional) Um tipo que indica ao usuário qual variável deve ser otimizada. |


# Problema Vectorizado (Problem)

In [1]:
import numpy as np
from pymoo.core.problem import Problem

class SphereWithConstraint(Problem):

    def __init__(self):
        super().__init__(n_var=10, n_obj=1, n_ieq_constr=1, xl=0.0, xu=1.0)

    def _evaluate(self, x, out, *args, **kwargs):
        out["F"] = np.sum((x - 0.5) ** 2, axis=1)  # Soma quadrática
        out["G"] = 0.1 - out["F"]  # Restrição radial


# Problema Elementwise (ElementwiseProblem)

In [2]:
import numpy as np
from pymoo.core.problem import ElementwiseProblem

class ElementwiseSphereWithConstraint(ElementwiseProblem):

    def __init__(self):
        xl = np.zeros(10)
        xl[0] = -5.0  # Primeiro elemento pode variar mais

        xu = np.ones(10)
        xu[0] = 5.0

        super().__init__(n_var=10, n_obj=1, n_ieq_constr=2, xl=xl, xu=xu)

    def _evaluate(self, x, out, *args, **kwargs):
        out["F"] = np.sum((x - 0.5) ** 2)  # Soma quadrática
        out["G"] = np.column_stack([0.1 - out["F"], out["F"] - 0.5])  # Duas restrições


# Problema Definido por Funções (FunctionalProblem)

In [3]:
import numpy as np
from pymoo.problems.functional import FunctionalProblem

# Definição das funções objetivo
objs = [
    lambda x: np.sum((x - 2) ** 2),  # Minimizar distância de x para 2
    lambda x: np.sum((x + 2) ** 2)   # Minimizar distância de x para -2
]

# Definição das restrições
constr_ieq = [
    lambda x: np.sum((x - 1) ** 2)  # Restrição quadrática
]

# Número de variáveis
n_var = 10

# Criando o problema
problem = FunctionalProblem(n_var,
                            objs,
                            constr_ieq=constr_ieq,
                            xl=np.array([-10, -5, -10]),
                            xu=np.array([10, 5, 10])
                            )

# Avaliação de 3 soluções aleatórias
F, G = problem.evaluate(np.random.rand(3, 10))

print(f"F: {F}\n")
print(f"G: {G}\n")


F: [[22.36140081 64.72622235]
 [21.59365944 68.10983732]
 [24.00863541 62.44837864]]

G: [[2.95260619]
 [3.22270391]
 [3.61857122]]



# Definindo Ótimos Conhecidos

In [4]:
class ZDT1(Problem):

    def __init__(self, n_var=30, **kwargs):
        super().__init__(n_var=n_var, n_obj=2, n_ieq_constr=0, xl=0, xu=1, vtype=float, **kwargs)

    def _calc_pareto_front(self, n_pareto_points=100):
        x = np.linspace(0, 1, n_pareto_points)
        return np.array([x, 1 - np.sqrt(x)]).T  # Define a Pareto-front conhecida

    def _evaluate(self, x, out, *args, **kwargs):
        f1 = x[:, 0]  # Primeira função objetivo
        g = 1 + 9.0 / (self.n_var - 1) * np.sum(x[:, 1:], axis=1)  # Função auxiliar
        f2 = g * (1 - np.power((f1 / g), 0.5))  # Segunda função objetivo

        out["F"] = np.column_stack([f1, f2])  # Retorna os valores de F
